# WOD predictor

## Objective

A WOD is a workout of the day. It is a specific, pre-written workout that is often used in CrossFit and other high-intensity interval training (HIIT) programs. The workout typically includes a variety of exercises, such as weightlifting, calisthenics, and cardio, and is designed to be challenging and to improve overall fitness.

The objective for this project is to train learning models to predict future WODs.

## WOD scraping

We are first scrapping WODs on Zenplanner.
The scrapping code is not super optimized there's still changes to be made.

/!\ May not work anymore if Zenplanner's DOM is changed.

In [3]:
import requests
from bs4 import BeautifulSoup
import numpy as np

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [ ]:
URL = ''

def parsePage(page):
    rest = False
    workout = ""
    date = ""
    soup = BeautifulSoup(page.content, 'html.parser')
    workout_elems = soup.find_all('div', class_='skillDesc')
    if len(workout_elems) > 0:
        lines = workout_elems[0].contents
        
        for line in lines:
            if str(line) != "<br/>":
                workout += str(line) + "\n"
        workout = workout[:-1]

    else:
        rest = True
        
    date_elems = soup.select('#idPage > legend > div:nth-child(1) > span')
    date = str(date_elems[0].text)
    
    link_elems = soup.select('#idPage > legend > div:nth-child(1) > a:nth-child(1)')
    previous = URL + str(link_elems[0]['href']) + '&FRAME=true'
    
    return workout, date, previous, rest

dates = []
workouts = []
while True:
    page = requests.get(URL)
    workout, date, URL, rest = parsePage(page)
    if not(rest):
        dates.append(date)
        workouts.append(workout)
    if date == "13/02/17": # Test end condition (this is the first workout posted)
        # TODO: rewrite end condition
        break
data = np.array([dates, workouts])
np.save('data', data)

## Load dataset

In [ ]:
data = np.load('data.npy')
dates = data[0]
workouts = data[1]

nb_workouts = workouts.shape[0]

f = open("dataset.txt", "w")
for i in range(nb_workouts):
    if len(workouts[i]) > 0:
        workout = workouts[i].replace("\n", "; ")
        if workouts[-1] == ' ':
            workout = workouts[:-1]
        f.write(workout + '\r\n')
f.close()

## Train network

In [74]:
from textgenrnn import textgenrnn

textgen = textgenrnn()
textgen.train_from_file('dataset.txt', num_epochs=5)

1,661 texts collected.
Training on 95,161 character sequences.
  ...
    to  
  ['...']
Train for 743 steps
Epoch 1/5
742/743 [============================>.] - ETA: 0s - loss: 1.4187####################
Temperature: 0.2
####################






####################
Temperature: 0.5
####################






####################
Temperature: 1.0
####################
TECH:; WOD: 7 Rounds; OHS; Z bend-usa burpee over walk

TECH: 2x eveimple push up; WOD: E0MOM 20x plank; ; 1 Rounds; 5 Rounds; -10x Band sit up; WAGG can 2min; 2x A/R shupe squat 60min;  7min EL Clean; 5x C2B sutches; / REST 6min; 10x squat 5 mil



743/743 [==============================] - 114s 153ms/step - loss: 1.4187
Epoch 2/5
742/743 [============================>.] - ETA: 0s - loss: 1.0838####################
Temperature: 0.2
####################
TECH: EMOM 15min; - 10x burpees; - 10x strict climbers 95/65#; WOD: AMRAP 7min; 10x sit up; 30x D-U; 10x clean and jerk 95/65#; 10x back squat; 15x strict clean and jerk 

## Prediction

In [89]:
workouts = textgen.generate(3, return_as_list=True)
for i in range(len(workouts)):
    print(workouts[i].replace("; ", "\n"))
    print('-----')

100%|██████████| 3/3 [00:13<00:00,  4.56s/it]

WOD: EMOM 20min
-10x squat clean and jerk 95/65
10-4-3-2-3
-----
TECH: EMOM 10min
-10x suicide
plate
-10x jumping lunge
-4x banded bar muscle up
- 1RM Squat
-10x squat snatch 155/105#
- extension hold
- press 95/65
- straight hold
- 10/10x cal row
-----
TECH: EMOM 15min 
- 10x Snade walking lunge
10x DB Push Press 95/65
10x C2B pull up
20x pull up
20x squat clean
10x push use 5
10x pull up
10x banded banded box jump 24/20
20x sit up
10x bar muscle up
30x D-U
20x pull up
-----


This is a very naive version of what a WOD predictor could be given a dataset of previous workouts.

Observations
- The model seems to invent new movements such as "straight hold"
- Some words seem to be repeated when they are never repeated in the original dataset
- It looks like the syntax of the original dataset needs to be cleaned a little more (e.g. presence or not of the character "-" at the beginning of a line
- There's too many movements per workout, which would never be the case in real workouts